In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import socket
import sys
import json
import matlab.engine
import numpy as np

eng = matlab.engine.start_matlab()

HOST = '192.168.0.92'
PORT = 3900

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind((HOST, PORT))
s.listen(5)

#action = {"MCS_DL": 17, "MCS_UL": 17, "max_DL_HARQ": 1, "max_UL_HARQ": 1, "Repetition": 1.0, "Grantfree_Ind": 0}
action = {"Repetition1": 1.0, "Repetition2": 1.0, "Repetition3": 1.0, "Repetition4": 1.0, "Repetition5": 1.0, 
          "MCS1": 28, "MCS2": 28, "MCS3": 28, "MCS4": 28, "MCS5": 28, 
          "HARQ1": 4, "HARQ2": 4, "HARQ3": 4, "HARQ4": 4, "HARQ5": 4}

print('server start at: %s:%s' % (HOST, PORT))
print('wait for connection...')
#s.settimeout(10)

while True:
    try:
        conn, addr = s.accept()
        print('connected by ' + str(addr))
        s.setblocking(False)
    except:
        pass

    while 1:
        print("start run")
        try:
            #indata = {'MCS_index':17, 'maxHARQ':2, 'Repetition':1.0}
            conn.settimeout(5)
            indata = conn.recv(1024)
            #conn.settimeout(5)
            if len(indata) == 0: # connection closed
                conn.close()
                print('client closed connection.')
                break
            print('recv: ' + indata.decode())
            try:
                action = eval(indata.decode())
                print(action)
                BLER, SINR, Latency, Throughput, DLUL = eng.WBTV2(action, nargout=5)
                BLER, SINR, Latency = np.array(BLER), np.array(SINR), np.array(Latency)

                reportt = '{\"BLER\":{'
                var = 0
                while var<len(BLER[0]):
                    reportt = reportt + '\"UE' + str(var+1) + '\":' + str(round(BLER[0][var],6)) + ','
                    var+=1

                reportt = reportt + '},\"SNR\":{'
                var1 = 0
                while var1<len(SINR[0]):
                    reportt = reportt + '\"UE' + str(var1+1) + '\":' + str(round(SINR[0][var1],3)) + ','
                    var1+=1

                reportt = reportt + '},\"Latency\":{'
                var2 = 0
                while var2<len(Latency[0]):
                    reportt = reportt + '\"UE' + str(var2+1) + '\":' + str(round(Latency[0][var2],3)) + ','
                    var2+=1

                reportt = reportt + '}'
                reportt = reportt + ',\"Throughput\":' + str(Throughput)
                reportt = reportt + '}'

                if DLUL == 1:
                    reportt = reportt + ',\"DL or UL\":\"DL\"}'
                else:
                    reportt = reportt + ',\"DL or UL\":\"UL\"}'

                print(reportt)
                data = json.dumps(reportt)
                try:
                    conn.sendall(bytes(data,encoding="utf-8"))
                except:
                    break
            except:
                continue

        except IOError:
            print ("action no change")
            print(action)
            BLER, SINR, Latency, Throughput, DLUL = eng.WBTV2(action, nargout=5)
            BLER, SINR, Latency = np.array(BLER), np.array(SINR), np.array(Latency)
            reportt = '{\"BLER\":{'
            var = 0
            while var<len(BLER[0]):
                reportt = reportt + '\"UE' + str(var+1) + '\":' + str(round(BLER[0][var],6)) + ','
                var+=1

            reportt = reportt + '},\"SNR\":{'
            var1 = 0
            while var1<len(SINR[0]):
                reportt = reportt + '\"UE' + str(var1+1) + '\":' + str(round(SINR[0][var1],3)) + ','
                var1+=1

            reportt = reportt + '},\"Latency\":{'
            var2 = 0
            while var2<len(Latency[0]):
                reportt = reportt + '\"UE' + str(var2+1) + '\":' + str(round(Latency[0][var2],3)) + ','
                var2+=1

            reportt = reportt + '}'
            reportt = reportt + ',\"Throughput\":' + str(Throughput)
            if DLUL == 1:
                reportt = reportt + ',\"DL or UL\":\"DL\"}'
            else:
                reportt = reportt + ',\"DL or UL\":\"UL\"}'
            
            print(reportt)
            data = json.dumps(reportt)
            try:
                conn.sendall(bytes(data,encoding="utf-8"))
            except:
                break

        
eng.quit()
        


server start at: 192.168.0.92:3900
wait for connection...
connected by ('192.168.0.92', 60213)
start run
action no change
{'Repetition1': 1.0, 'Repetition2': 1.0, 'Repetition3': 1.0, 'Repetition4': 1.0, 'Repetition5': 1.0, 'MCS1': 28, 'MCS2': 28, 'MCS3': 28, 'MCS4': 28, 'MCS5': 28, 'HARQ1': 4, 'HARQ2': 4, 'HARQ3': 4, 'HARQ4': 4, 'HARQ5': 4}
{"BLER":{"UE1":0.07,"UE2":0.07,"UE3":0.22,"UE4":0.032,"UE5":0.12,},"SNR":{"UE1":23.333,"UE2":23.333,"UE3":20.333,"UE4":24.667,"UE5":21.667,},"Latency":{"UE1":8.0,"UE2":7.0,"UE3":8.0,"UE4":5.0,"UE5":6.0,},"Throughput":1280.0,"DL or UL":"DL"}
start run
action no change
{'Repetition1': 1.0, 'Repetition2': 1.0, 'Repetition3': 1.0, 'Repetition4': 1.0, 'Repetition5': 1.0, 'MCS1': 28, 'MCS2': 28, 'MCS3': 28, 'MCS4': 28, 'MCS5': 28, 'HARQ1': 4, 'HARQ2': 4, 'HARQ3': 4, 'HARQ4': 4, 'HARQ5': 4}
{"BLER":{"UE1":0.012,"UE2":0.05,"UE3":0.12,"UE4":0.05,"UE5":0.12,},"SNR":{"UE1":27.0,"UE2":23.667,"UE3":22.0,"UE4":24.0,"UE5":22.167,},"Latency":{"UE1":9.0,"UE2":9.0,"U

action no change
{'Repetition1': 1.0, 'Repetition2': 1.0, 'Repetition3': 1.0, 'Repetition4': 1.0, 'Repetition5': 1.0, 'MCS1': 28, 'MCS2': 28, 'MCS3': 28, 'MCS4': 28, 'MCS5': 28, 'HARQ1': 4, 'HARQ2': 4, 'HARQ3': 4, 'HARQ4': 4, 'HARQ5': 4}
{"BLER":{"UE1":0.02,"UE2":0.12,"UE3":0.12,"UE4":0.0052,"UE5":0.16,},"SNR":{"UE1":26.333,"UE2":21.667,"UE3":21.667,"UE4":28.333,"UE5":21.0,},"Latency":{"UE1":7.0,"UE2":6.0,"UE3":33.0,"UE4":8.0,"UE5":5.0,},"Throughput":1024.0,"DL or UL":"DL"}
start run
action no change
{'Repetition1': 1.0, 'Repetition2': 1.0, 'Repetition3': 1.0, 'Repetition4': 1.0, 'Repetition5': 1.0, 'MCS1': 28, 'MCS2': 28, 'MCS3': 28, 'MCS4': 28, 'MCS5': 28, 'HARQ1': 4, 'HARQ2': 4, 'HARQ3': 4, 'HARQ4': 4, 'HARQ5': 4}
{"BLER":{"UE1":0.16,"UE2":0.05,"UE3":0.05,"UE4":0.032,"UE5":0.032,},"SNR":{"UE1":20.667,"UE2":23.667,"UE3":23.667,"UE4":25.0,"UE5":25.333,},"Latency":{"UE1":8.0,"UE2":5.0,"UE3":9.0,"UE4":8.0,"UE5":6.0,},"Throughput":1280.0,"DL or UL":"DL"}
start run
